Mam prośbę o przygotowanie dwóch widgetów jupyterowskich.

In [5]:
import geopandas as gpd
from shapely.geometry import Point, Polygon
from rasterstats import zonal_stats

# Wybieranie punktów na mapie


In [10]:
def define_reference_temperatures(geotiff_path) -> gpd.GeoDataFrame:
    # 1. Wyśietl na mapie leaflet GeoTiff ze ścieżki.
    # 2. Widget ma tylko mieć guziki do dodawania i usuwania punktów (bez poligonów itp., te mogą być ukryte). 
    # 3. Użytkownik tworzy dowolną ilość punktów na mapie.
    # 4. Po każdorazowym utworzeniu punktu ma się wyświetlać popup z polem tekstowym do wpisywania liczby float z labelką "Temperature" do potwierdzenia guzikiem OK lub enterem.
    # 5. W prawym dolnym rogu guzik "CONTINUE", który kończy działanie widgetu.
    # 6. Zwrócony zostaje obiekt geopandas.GeoDataFrame z punktami i temperaturami każdego z nich.

    example_gdf = gpd.GeoDataFrame(geometry=[Point(19.11274, 50.93590), Point(19.11458, 50.93773)], data={'temperature': [8.1, 7.9]}, crs='EPSG:4326')
    return example_gdf

In [16]:
ref_points = define_reference_temperatures("data/output/mosaic.tiff")
footprints = gpd.read_file("data/output/footprints.geojson")
ref_points = ref_points.to_crs(footprints.crs)
ref_points["footprints"] = None
for id_i, ref_point in ref_points.iterrows():
    #get footprints that intersect with the reference point
    id_js = []
    for id_j, footprint in footprints.iterrows():
        if footprint['geometry'].intersects(ref_point['geometry']):
            id_js.append(id_j)
    ref_points.at[id_i, "footprints"] = id_js

# Zaznaczanie poligonów na serii rastrów

In [ ]:
def probe_temperature(polygons: gpd.GeoDataFrame, geotiff_path: str) -> float:
    """Probes temperature on raster for a given polygon using zonal statistics."""
    stats = zonal_stats(polygons['geometry'], geotiff_path, stats="mean")
    means = []
    for stat in stats:
        means.append(stat['mean'])
    return sum(means)/len(means)

def define_reference_areas(ref_points: gpd.GeoDataFrame):
    footprints["offset"] = None
    # 1. Wyświetl mape leaflet
    for id_i, ref_point in ref_points.iterrows():
        for id_j in ref_point["footprints"]:
            footprint = footprints.loc[id_j]
            geotiff_path = f"data/output/geotiff/{footprint['name']}"
            # 2. Wyświetl na mapie leaflet GeoTIFF ze ścieżki geotiff_path
            # 3. Widget ma tylko mieć guziki do dodawania i usuwania poligonów (bez punktów itp., te mogą być ukryte).
            # 4. Użytkownik tworzy dowolną ilość poligonów na mapie. Tu brak popupów.
            # 5. Użytkownik akceptuje poligony guzikiem "CONTINUE" w prawym dolnym rogu. Zwracane są w postaci geopandas.GeoDataFrame, np:
            example_polygons = gpd.GeoDataFrame(geometry=[Polygon([(19.11274, 50.93590), (19.11458, 50.93773), (19.11458, 50.93590)])], crs='EPSG:4326')
            temperature = probe_temperature(example_polygons, geotiff_path)
            offset = temperature - ref_point['temperature']
            footprints.at[id_j, "offset"] = offset #aktaulizacja offsetu dla danego footprintu
            # 6. Geotiff i poligony zostają usunięte z mapy, by zrobić miejsce na następny podczas kolejnej iteracji.
            

In [11]:

define_reference_areas(ref_points)